In [348]:
import pandas
import threading
import requests
import json 
import time
Mutex = threading.Lock()

In [349]:
def PokemonAPIEndPoint(*vargs, **kwargs)->dict:
    thread = threading.current_thread()
    curl = 'https://pokeapi.co/api/v2/pokemon?'
    if vargs == ():
        pass
    else:
        # curl += vargs
        pass

    if kwargs == {}:
        pass
    else:
        curl = curl +  ''.join([f'&{x}={kwargs[x]}' for x in kwargs])
    response = requests.get(curl)
    json_content = json.loads(response.content)
    df = pandas.DataFrame(json_content['results'])
    apiCallCount = None
    with Mutex:
        try:
            with open('countApiCalls.txt', 'r+') as file:
                temp  = [line.replace('\x00','') for line in file.readlines()][0]
                # print(temp, 'is temp')
                if temp == '':
                    apiCallCount = 0
                    file.truncate(0)
                    file.write('0')
                    file.close()
                else:
                    # file.truncate()
                    apiCallCount = float(temp) + 1
                    file.truncate(0)

                    file.write(str(apiCallCount))
                    file.close()
                    pass
                print(apiCallCount, 'is call count')
                
        except:
            with open('countApiCalls.txt', 'w+') as file:
                file.write('0')
        time.sleep(1)
    # print(df)
    return dict(json.loads(response.content))
    

In [350]:
class ThreadedQuerry:
    def __init__(self) -> None:
        self.threads  = []
    def JoinAllThreads(self) -> None:
        for thread in self.threads:
            if getattr(thread, "shouldRun", None) != None:
                setattr(thread,"shouldRun", not thread.shouldRun)
            thread.join()
        for thread in self.threads:
            del thread
        self.threads = []
    def LoadFrom(self,*vargs, **kwargs) -> None:
        for x in vargs:
            args  = kwargs
            args['offset'] = x 
            thread = threading.Thread(target=PokemonAPIEndPoint, args = (), kwargs=args)
            thread.shouldRun = False
            thread.start()
            self.threads.append(thread)
        self.JoinAllThreads()

In [351]:
QuerryWrapper = ThreadedQuerry()

In [352]:
QuerryWrapper.LoadFrom(0 , 200 , 400 , limit = 200)

10.0 is call count
11.0 is call count
12.0 is call count


In [353]:
QuerryWrapper.threads


[]